In [1]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [37]:
import torch

from mvn.utils.minimon import MiniMon
from mvn.utils.misc import flush_cache

from mvn.ipynb import get_args, get_config, build_labels
from train import setup_dataloaders, setup_experiment, build_env, do_train


flush_cache()

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |   60654 KB |     768 MB |  104901 MB |  104842 MB |
|       from large pool |   55040 KB |     707 MB |  101279 MB |  101225 MB |
|       from small pool |    5614 KB |      69 MB |    3622 MB |    3616 MB |
|---------------------------------------------------------------------------|
| Active memory         |   60654 KB |     768 MB |  104901 MB |  104842 MB |
|       from large pool |   55040 KB |     707 MB |  101279 MB |

In [38]:
is_distributed = False
master = True
device = torch.device(0)
print('using dev {}'.format(device))
    
args = get_args()

# just to get a feeling of the dataset
# labels, mask, indices = build_labels(config.dataset.train.labels_path, 10000)
# labels, mask, indices = build_labels(config.dataset.train.labels_path, 10, allowed_subjects=['S9', 'S11'])

using dev cuda:0
# available GPUs: 1


In [44]:
config = get_config(args)
do_train(None, None, config, device, is_distributed, master)

  training dataset length: 12
  validation dataset length: 5
Experiment name: human36m_alg_AlgebraicTriangulationNet@30.04.2021-18:36:22
epoch    0 has started!
tensor(2.3812, device='cuda:0', grad_fn=<AddBackward0>)
   tensor(8.9431, device='cuda:0', grad_fn=<AddBackward0>)
tensor(11.1730, device='cuda:0', grad_fn=<AddBackward0>)
   tensor(18.7223, device='cuda:0', grad_fn=<AddBackward0>)
tensor(21.4103, device='cuda:0', grad_fn=<AddBackward0>)
   tensor(29.4229, device='cuda:0', grad_fn=<AddBackward0>)
tensor(32.0396, device='cuda:0', grad_fn=<AddBackward0>)
   tensor(39.9246, device='cuda:0', grad_fn=<AddBackward0>)
tensor(42.5908, device='cuda:0', grad_fn=<AddBackward0>)
   tensor(49.5100, device='cuda:0', grad_fn=<AddBackward0>)
tensor(51.8693, device='cuda:0', grad_fn=<AddBackward0>)
   tensor(58.6122, device='cuda:0', grad_fn=<AddBackward0>)
tensor(61.3207, device='cuda:0', grad_fn=<AddBackward0>)
   tensor(68.8271, device='cuda:0', grad_fn=<AddBackward0>)
tensor(71.3609, device